In [1]:
import os
os.chdir(f"/home/{os.getlogin()}/watttime-python-client-aer-algo")

import math
import numpy as np
import pandas as pd
import datetime
import pytz
from datetime import datetime, timedelta
from watttime import WattTimeForecast, WattTimeHistorical

import optimizer.s3 as s3u
import evaluation.eval_framework as efu

import watttime.shared_anniez.alg.optCharger as optC
import watttime.shared_anniez.alg.moer as Moer

region = "ERCOT_AUSTIN"
username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

actual_data = WattTimeHistorical(username, password)
hist_data = WattTimeForecast(username, password)

s3 = s3u.s3_utils()
key = '20240715_1k_synth_users.csv'
generated_data = s3.load_csvdataframe(file=key)

# Synthetic User Data

In [2]:
synth_data = generated_data.copy(deep=True)

synth_data["plug_in_time"] = pd.to_datetime(synth_data["plug_in_time"])
synth_data["unplug_time"] = pd.to_datetime(synth_data["unplug_time"])

In [3]:
# one user - 10 sessions - Feb 2024
synth_data = synth_data.query('user_type == "r4.736000000000001_tc95_avglc23868_sdlc7685"').sample(10)

## API Data

In [4]:
synth_data['moer_data'] = synth_data.apply(
    lambda x: efu.get_historical_fcst_data(
    x.plug_in_time,
    math.ceil(x.total_intervals_plugged_in),
    region = region
    ), axis = 1
)

synth_data['moer_data_actual'] = synth_data.apply(
    lambda x: efu.get_historical_actual_data(
    x.plug_in_time,
    math.ceil(x.total_intervals_plugged_in),
    region = region
    ), axis = 1
)

## MOER - No Optimization - Actual Emissions

In [5]:
synth_data['charger_baseline_actual'] = synth_data.apply(
    lambda x: efu.get_schedule_and_cost(
        x.MWh_fraction,
        x.charged_kWh_actual / 1000,
        math.ceil(x.total_intervals_plugged_in), # will throw an error if the plug in time is too shart to reach full charge, should soften to a warning
        x.moer_data_actual,
        asap = True
        ), 
        axis = 1
        )

synth_data['baseline_actual_emissions'] = synth_data['charger_baseline_actual'].apply(
    lambda  x: x.get_total_cost()
    )

In [6]:
synth_data['baseline_actual_emissions']

13    59.104096
1     24.174517
11    45.562688
6     33.462997
5     13.597845
3     16.434315
14    16.949749
8     44.940693
0     25.370752
2     38.100331
Name: baseline_actual_emissions, dtype: float64

## MOER - Simple Optimization - Forecast Emissions

In [7]:
synth_data['charger_simple_forecast']= synth_data.apply(
    lambda x: efu.get_schedule_and_cost(
        x.MWh_fraction,
        x.charged_kWh_actual / 1000,
        math.ceil(x.total_intervals_plugged_in), # will throw an error if the plug in time is too shart to reach full charge, should soften to a warning
        x.moer_data,
        asap = False
        ), 
        axis = 1
        )

synth_data['simple_estimated_emissions'] = synth_data['charger_simple_forecast'].apply(
    lambda  x: x.get_total_cost()
    )

In [29]:
def get_total_emission(moer, schedule): 
    x = np.array(schedule).flatten()
    return np.dot(moer[:x.shape[0]], x)

In [31]:
synth_data.apply(
    lambda x: get_total_emission(
        x.moer_data_actual['value'],
        x.charger_simple_forecast.get_schedule()
    ),
    axis = 1
)

13    59.104096
1     24.174517
11    45.562688
6     33.462997
5     13.597845
3     16.434315
14    16.949749
8     44.940693
0     25.370752
2     38.100331
dtype: float64